# Prepare PPI Network

In [7]:
import pandas as pd
import numpy as np

In [8]:
interactions = pd.DataFrame.from_csv('ConsensusPathDB_human_PPI.csv', header=1, sep='\t', encoding='utf8')

In [9]:
interactions_nona = interactions.dropna()

## Get  all proteins involvec in interactions

In [10]:
all_proteins = []
count = 0
for index, row in interactions_nona.iterrows():
    if row.interaction_confidence > 0.5:
        for prot in row.interaction_participants.split(','):
            all_proteins.append(prot.strip())
    count += 1
    if count % 10000 == 0:
        print ("Processed {} rows".format(count))
print ("Processed {} interactions involving {} different proteins".format(len(all_proteins), len(set(all_proteins))))

Processed 10000 rows
Processed 20000 rows
Processed 30000 rows
Processed 40000 rows
Processed 50000 rows
Processed 60000 rows
Processed 70000 rows
Processed 80000 rows
Processed 90000 rows
Processed 100000 rows
Processed 110000 rows
Processed 120000 rows
Processed 130000 rows
Processed 140000 rows
Processed 150000 rows
Processed 160000 rows
Processed 170000 rows
Processed 180000 rows
Processed 190000 rows
Processed 200000 rows
Processed 210000 rows
Processed 220000 rows
Processed 230000 rows
Processed 240000 rows
Processed 250000 rows
Processed 260000 rows
Processed 270000 rows
Processed 356690 interactions involving 12762 different proteins


## Construct Adjacency Matrix

In [11]:
all_proteins_unique = set(all_proteins)
N = len(all_proteins_unique)
adjacency_matrix = np.zeros((N, N), np.uint8)
adj_df = pd.DataFrame(adjacency_matrix, index=all_proteins_unique, columns=all_proteins_unique)

In [12]:
count = 0
for index, row in interactions_nona.iterrows():
    if row.interaction_confidence > 0.5:
        interaction_partners = row.interaction_participants.split(',')
        if len(interaction_partners) == 2:
            adj_df.ix[interaction_partners[0], interaction_partners[1]] = 1
            adj_df.ix[interaction_partners[1], interaction_partners[0]] = 1
            
    count += 1
    if count % 10000 == 0:
        print ("Processed {} out of {} rows".format(count, interactions_nona.shape[0]))

Processed 10000 out of 272744 rows
Processed 20000 out of 272744 rows
Processed 30000 out of 272744 rows
Processed 40000 out of 272744 rows
Processed 50000 out of 272744 rows
Processed 60000 out of 272744 rows
Processed 70000 out of 272744 rows
Processed 80000 out of 272744 rows
Processed 90000 out of 272744 rows
Processed 100000 out of 272744 rows
Processed 110000 out of 272744 rows
Processed 120000 out of 272744 rows
Processed 130000 out of 272744 rows
Processed 140000 out of 272744 rows
Processed 150000 out of 272744 rows
Processed 160000 out of 272744 rows
Processed 170000 out of 272744 rows
Processed 180000 out of 272744 rows
Processed 190000 out of 272744 rows
Processed 200000 out of 272744 rows
Processed 210000 out of 272744 rows
Processed 220000 out of 272744 rows
Processed 230000 out of 272744 rows
Processed 240000 out of 272744 rows
Processed 250000 out of 272744 rows
Processed 260000 out of 272744 rows
Processed 270000 out of 272744 rows


In [23]:
np.sum(adj_df).sum()

356690

## Store Adjacency matrix on Disk

In [14]:
f = h5py.File('ppi_networks.h5', 'w')
f.create_dataset('consensusPathDB_ppi', data=adj_df, shape=adj_df.shape)
f.close()